In [11]:
import numpy as np

from gymnasium import utils
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium.error import Error
from gymnasium.spaces import Box
from gymnasium.utils.env_checker import check_env

In [12]:
class QRPfRA_v3(MujocoEnv, utils.EzPickle):
    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
    }

    def __init__(self, xml_file="/Users/deniz/PycharmProjects/QRPfRA_Senior_Project/QRPfRA/IK_Models/qrpfra_v3_leg_ik_scene_left.xml", frame_skip=1, **kwargs):
        utils.EzPickle.__init__(self, xml_file, frame_skip, **kwargs)

        MujocoEnv.__init__(
            self,
            xml_file,
            frame_skip=frame_skip,
            observation_space=None,  # needs to be defined after
            default_camera_config={},
            **kwargs,
        )

        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

        obs_size = self.data.qpos.size + self.data.qvel.size

        self.observation_space = Box(
            low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64
        )

    def step(self, action):
        x_position_before = self.data.qpos[0]

        self.do_simulation(action, self.frame_skip)
        x_position_after = self.data.qpos[0]

        observation = self._get_obs()
        reward = x_position_after - x_position_before
        info = {}

        if self.render_mode == "human":
            self.render()

        return observation, reward, False, False, info

    def _get_obs(self):
        #position = self.data.qpos.flat.copy()
        #velocity = self.data.qvel.flat.copy()
        sensor_data = self.data.sensordata.flat.copy()

        return sensor_data

    def reset_model(self):
        qpos = self.init_qpos
        qvel = self.init_qvel
        self.set_state(qpos, qvel)

        observation = self._get_obs()

        return observation

    def _get_reset_info(self):
        return {"works": True}



In [13]:
env = QRPfRA_v3()
check_env(env)
obs = env.reset()
print(obs)


(array([-0.045 ,  0.1335, -0.107 ,  0.    ,  0.    ,  0.    ]), {'works': True})


In [14]:
left_leg_dataset = [] # [obs0, obs1, obs2, act0, act1, act2]

In [15]:
for i in range(-100, 100):
    for j in range(-100, 100):
        for k in range(-100, 100):
            #env.render_mode = "human"
            env_obs = []
            for step_cnt in range(0,12):
                obs, reward, done, _, info = env.step([float(m) for m in [i, j, k]])
                if step_cnt == 11:
                    env_obs = obs

            left_leg_dataset.append([env_obs[0], env_obs[1], env_obs[2], i/100, j/100, k/100])

            env.reset()

env.close()

In [16]:
print(len(left_leg_dataset))

8000000


In [17]:
#Convert left leg list to numpy array
left_leg_dataset_as_np = np.array(left_leg_dataset)

In [18]:
print(left_leg_dataset_as_np[0:4])

observations = left_leg_dataset_as_np[:, 0:3][:]
print(observations)
actions = left_leg_dataset_as_np[:, 3:][:]
print(actions)

[[ 0.13255586 -0.14628923 -0.11752452 -1.         -1.         -1.        ]
 [ 0.13295846 -0.14548642 -0.11775573 -1.         -1.         -0.99      ]
 [ 0.13349795 -0.14438762 -0.11806557 -1.         -1.         -0.98      ]
 [ 0.13402739 -0.14328233 -0.11836963 -1.         -1.         -0.97      ]]
[[ 0.13255586 -0.14628923 -0.11752452]
 [ 0.13295846 -0.14548642 -0.11775573]
 [ 0.13349795 -0.14438762 -0.11806557]
 ...
 [ 0.02261306  0.04277707  0.0758465 ]
 [ 0.02209332  0.04166701  0.07554264]
 [ 0.02156351  0.04056336  0.0752329 ]]
[[-1.   -1.   -1.  ]
 [-1.   -1.   -0.99]
 [-1.   -1.   -0.98]
 ...
 [ 0.99  0.99  0.97]
 [ 0.99  0.99  0.98]
 [ 0.99  0.99  0.99]]


In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_obs, test_obs, train_actions, test_actions = train_test_split(observations, actions, test_size=0.1, shuffle=True)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='tanh')
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train the model
model.fit(train_obs, train_actions, epochs=10, batch_size=1024)

# Evaluate the model on the testing set
test_loss = model.evaluate(test_obs, test_actions)
print('Test Loss:', test_loss)


NameError: name 'observations' is not defined

In [21]:
model.save("left_legs_model", overwrite=True)

INFO:tensorflow:Assets written to: left_legs_model/assets


INFO:tensorflow:Assets written to: left_legs_model/assets
